# Domain Prompts and Chat GPT Instructions

## General Instructions

Goal: We need to make sure the robot has a sufficiently complex model of the scenario to perform tasks safely.

The robot's model consists of states (symbolic literals or lists of mutually exclusive (mutex) states) and actions (name, pre-condition states, added post-condition states, and removed post-condition states). You will be prompted with questions to explore the robot's model and provide improvements.

Here's the robot's model (formatted as a YAML file):

## Lunar Habitat Airlock

Scenario: A robot is on the lunar surface, assisting astronauts living in a habitat with science experiments and space exploration tasks. The pressurized lunar habitat has an airlock, with two doors on either side. One door from the pressurized habitat to the airlock chamber, one door from the airlock chamber to the lunar surface.

In [ ]:
domains = ["lunar_habitat_airlock"]

# Colab/Drive Setup

In [2]:
from google.colab import drive
from google.colab import runtime
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/Object\ Affordances/HRRT/human_robot_red_teaming
%ls

Mounted at /content/gdrive
/content/gdrive/MyDrive/Object Affordances/HRRT/human_robot_red_teaming
config/  docs/  hrrt.ipynb  README.md  scripts/


In [3]:
# %cd ../
# %ls
# %cd human_robot_red_teaming
%cd scripts/
%ls

/content/gdrive/MyDrive/Object Affordances/HRRT/human_robot_red_teaming/scripts
hrrt2.py  hrrt3.py  hrrt4.py  __pycache__/  yaml_formatting_checks.py


# HRRT Imports

In [4]:
from yaml_formatting_checks import KBFormatting, ModelFormatting
from hrrt2 import hrrt2, process_chatbot_hrrt2_text, write_hrrt2_yaml
from hrrt3 import hrrt3, write_hrrt3_yaml
from hrrt4 import hrrt4, write_hrrt4_yaml

# Test Knowledge Base and Model Formatting

In [112]:
# from yaml_formatting_checks import KBFormatting, ModelFormatting

kb_file = "../config/lunar_habitat_airlock/knowledge_base.yaml"
model_file = "../config/lunar_habitat_airlock/model0.yaml"

success, kb = KBFormatting.format_kb(kb_file)
if success:
  print("KNOWLEDGE BASE READING SUCCESSFUL!")
else:
  print("Knowledge base reading failed :(")

succes, model = ModelFormatting.format_model(model_file)
if success:
  print("MODEL READING SUCCESSFUL!")
else:
  print("Model reading failed :(")

KNOWLEDGE BASE READING SUCCESSFUL!
MODEL READING SUCCESSFUL!


# HRRT Utility Testing: Interactive Inputs

## Test HRRT Level 2: Enumerate Possibilities

* **Inputs**: model (states and actions)
* **Outputs**: possibilities (*s*,*a*,*sp*) for states *s* within which action *a* can be taken to achieve next state *sp*
* **Effects**: query blue team for validity of possible state

In [ ]:
# from hrrt2 import hrrt2, write_hrrt2_yaml

possibilities = hrrt2(model, interactive=True)
print("Found " + str(len(possibilities)) + " possibilities on level HRRT2!")

write_hrrt2_yaml(possibilities, model_file)

print("LEVEL HRRT2 SUCCESSFUL!")

Found possible state: ['robot_inside_habitat', 'door_habitat_airlock_unlocked_opened']
    within which it is possible to take action enter_airlock_from_habitat
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) y

Found possible state: ['robot_inside_airlock', 'door_habitat_airlock_unlocked_opened']
    within which it is possible to take action enter_habitat_from_airlock
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) y

Found possible state: ['robot_inside_airlock', 'door_airlock_surface_unlocked_opened']
    within which it is possible to take action enter_surface_from_airlock
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) y

Found possible state: ['robot_outside_habitat', 'door_airlock_surface_unlocked_opened']
    within which it is possible to take action enter_airlock_from_surface
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) y

Found possible state: ['robot_inside_habitat', 'robot_has_key', 'door_habit

## Test HRRT Level 3: Assumptions

* **Inputs**: model (states and actions)
* **Outputs**: assumptions (*a*,*s*) about whether pre-condition or post-condition state *s* can be achieved for/from action *a*
* **Effects**: query blue team for validity of assumption

In [ ]:
# from hrrt3 import hrrt3, write_hrrt3_yaml

assumptions = hrrt3(model, interactive=True)
pre_assump, post_add_assump, post_sub_assump = assumptions
print("Found " + str(len(pre_assump) + len(pre_assump) + len(pre_assump)) + " assumptions on level HRRT3!")

write_hrrt3_yaml(assumptions, model_file)

print("LEVEL HRRT3 SUCCESSFUL!")

Implicit assumption that pre-condition state robot_inside_airlock can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) y

Implicit assumption that pre-condition state robot_has_key can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) n

Implicit assumption that pre-condition state door_habitat_airlock_locked_closed can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) y

Implicit assumption that pre-condition state robot_outside_habitat can be achieved to perform action unlock_open_door_airlock_surface
    Is this assumption always valid? (Y/N) y

Implicit assumption that pre-condition state robot_has_key can be achieved to perform action unlock_open_door_airlock_surface
    Is this assumption always valid? (Y/N) y

Implicit assumption that pre-condition state door_airlock_surface_locked_closed can be achieved to perfo

## Test HRRT Level 4: Update Model Knowledge

* **Inputs**: model (states and actions), possibilities (HRRT2), assumptions (HRRT3)
* **Outputs**: model update hypothesis (*S+*, *S-*, *A+*, *A-*) for states/actions to add/remove from model
* **Effects**: query blue team and red team for model updates, prompt for team reflections about the problem domain

In [ ]:
# from hrrt4 import hrrt4, write_hrrt4_yaml

updated_kb, updated_model, model_hypothesis, facts = hrrt4(kb, model, possibilities, assumptions, interactive=True)
states_add, states_sub, actions_add, actions_sub = model_hypothesis
print("Found " + str(len(states_add) + len(states_sub) + len(actions_add) + len(actions_sub)) + " model modifications on level HRRT4!")
print("Found " + str(len(facts)) + " team reflections on level HRRT4!")

write_hrrt4_yaml(updated_kb, updated_model,
                 model_hypothesis, facts,
                 kb_file, model_file)

print("LEVEL HRRT4 SUCCESSFUL!")


==================== HRRT LEVEL 2 REVIEW ====================

The following possibilities were identified as invalid on Level HRRT2:
    possible state: ['robot_inside_habitat', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_unlocked_opened']
        within which it is possible to take action enter_airlock_from_habitat

Let's update the model to address these invalid possibilities!

What states should be added to the model?
    Recall symbolic states are literals or lists of mutually exclusive (mutex) literals.
    Please enter states to add to model (or Q to quit)
add state (or list of comma separated mutex states): both_doors_unlocked

add state (or list of comma separated mutex states): q

What states should be removed from the model?
    Recall symbolic states are literals or lists of mutually exclusive (mutex) literals.
    Please enter states to remove from model (or Q to quit)
remove state (or list of comma separated mutex states): q

What actions should be adde

# HRRT Utility Testing: Transcript for ChatGPT Blue Team

## Test HRRT Level 2: Enumerate Possibilities

In [5]:
possibilities = hrrt2(model, interactive=False)
print("Found " + str(len(possibilities)) + " possibilities on level HRRT2!")
print("[***** GET INPUT FROM CHAT GPT BLUE TEAM *****]")
print("LEVEL HRRT2 SUCCESSFUL!")

Found possible state: ['robot_inside_habitat', 'door_habitat_airlock_unlocked_opened']
    within which it is possible to take action enter_airlock_from_habitat
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) __________

Found possible state: ['robot_inside_airlock', 'door_habitat_airlock_unlocked_opened']
    within which it is possible to take action enter_habitat_from_airlock
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) __________

Found possible state: ['robot_inside_airlock', 'door_airlock_surface_unlocked_opened']
    within which it is possible to take action enter_surface_from_airlock
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) __________

Found possible state: ['robot_outside_habitat', 'door_airlock_surface_unlocked_opened']
    within which it is possible to take action enter_airlock_from_surface
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) __________

Found possible state: ['robot_inside_ha

## Test HRRT Level 3: Assumptions

In [91]:
assumptions = hrrt3(model, interactive=False)
pre_assump, post_add_assump, post_sub_assump = assumptions
print("Found " + str(len(pre_assump) + len(pre_assump) + len(pre_assump)) + " assumptions on level HRRT3!")
print("[***** GET INPUT FROM CHAT GPT BLUE TEAM *****]")
print("LEVEL HRRT3 SUCCESSFUL!")

Implicit assumption that pre-condition state robot_inside_airlock can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) __________

Implicit assumption that pre-condition state robot_has_key can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) __________

Implicit assumption that pre-condition state door_habitat_airlock_locked_closed can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) __________

Implicit assumption that pre-condition state robot_outside_habitat can be achieved to perform action unlock_open_door_airlock_surface
    Is this assumption always valid? (Y/N) __________

Implicit assumption that pre-condition state robot_has_key can be achieved to perform action unlock_open_door_airlock_surface
    Is this assumption always valid? (Y/N) __________

Implicit assumption that pre-condition state door_airlock_s

## Test HRRT Level 4: Update Model Knowledge

In [7]:
updated_kb, updated_model, model_hypothesis, facts = hrrt4(kb, model, possibilities, assumptions, interactive=False)
states_add, states_sub, actions_add, actions_sub = model_hypothesis
print("Found " + str(len(states_add) + len(states_sub) + len(actions_add) + len(actions_sub)) + " model modifications on level HRRT4!")
print("Found " + str(len(facts)) + " team reflections on level HRRT4!")
print("[***** GET INPUT FROM CHAT GPT BLUE TEAM *****]")
print("LEVEL HRRT4 SUCCESSFUL!")


==================== HRRT LEVEL 2 REVIEW ====================

The following possibilities were identified as invalid on Level HRRT2:
    possible state: ['robot_inside_habitat', 'door_habitat_airlock_unlocked_opened']
        within which it is possible to take action enter_airlock_from_habitat

    possible state: ['robot_inside_airlock', 'door_habitat_airlock_unlocked_opened']
        within which it is possible to take action enter_habitat_from_airlock

    possible state: ['robot_inside_airlock', 'door_airlock_surface_unlocked_opened']
        within which it is possible to take action enter_surface_from_airlock

    possible state: ['robot_outside_habitat', 'door_airlock_surface_unlocked_opened']
        within which it is possible to take action enter_airlock_from_surface

    possible state: ['robot_inside_habitat', 'robot_has_key', 'door_habitat_airlock_unlocked_opened']
        within which it is possible to take action enter_airlock_from_habitat

    possible state: ['robot

# ChatGPT Testing

## HRRT Level 2

### Prompts

Please fill in the blanks with [Y/N] for yes/no to answer the following questions:

In [67]:
possibilities = hrrt2(model, interactive=False)

Found possible state: ['robot_inside_habitat', 'door_habitat_airlock_unlocked_opened']
    within which it is possible to take action enter_airlock_from_habitat
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) __________

Found possible state: ['robot_inside_airlock', 'door_habitat_airlock_unlocked_opened']
    within which it is possible to take action enter_habitat_from_airlock
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) __________

Found possible state: ['robot_inside_airlock', 'door_airlock_surface_unlocked_opened']
    within which it is possible to take action enter_surface_from_airlock
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) __________

Found possible state: ['robot_outside_habitat', 'door_airlock_surface_unlocked_opened']
    within which it is possible to take action enter_airlock_from_surface
    Is this possibility always valid (safe, feasible, etc.)? (Y/N) __________

Found possible state: ['robot_inside_ha

### ChatGPT's Responses

In [68]:
hrrt2_raw_text = '''Here are the answers based on the current model and potential safety concerns, especially around airlock pressurization and the risk of both doors being open simultaneously:

1. **['robot_inside_habitat', 'door_habitat_airlock_unlocked_opened']** - **N** (Risk if the outer door is also open, causing depressurization.)
2. **['robot_inside_airlock', 'door_habitat_airlock_unlocked_opened']** - **N** (Unsafe if the outer door is open.)
3. **['robot_inside_airlock', 'door_airlock_surface_unlocked_opened']** - **N** (Unsafe if the inner door is open—depressurization risk.)
4. **['robot_outside_habitat', 'door_airlock_surface_unlocked_opened']** - **N** (Unsafe if the inner door is open simultaneously.)
5. **['robot_inside_habitat', 'robot_has_key', 'door_habitat_airlock_unlocked_opened']** - **N** (Same depressurization risk as #1.)
6. **['robot_inside_airlock', 'robot_has_key', 'door_habitat_airlock_locked_closed']** - **Y** (Safe as both doors are controlled, and no airlock breach risk.)
7. **['robot_inside_airlock', 'robot_has_key', 'door_habitat_airlock_unlocked_opened']** - **N** (Unsafe if outer door is open simultaneously.)
8. **['robot_inside_airlock', 'robot_has_key', 'door_airlock_surface_unlocked_opened']** - **N** (Unsafe if inner door is open simultaneously.)
9. **['robot_outside_habitat', 'robot_has_key', 'door_airlock_surface_locked_closed']** - **Y** (Safe; robot is outside with door locked.)
10. **['robot_outside_habitat', 'robot_has_key', 'door_airlock_surface_unlocked_opened']** - **N** (Unsafe if inner door is also open.)
11. **['robot_inside_habitat', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_locked_closed']** - **Y** (Safe; outer door is closed.)
12. **['robot_inside_habitat', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_unlocked_opened']** - **N** (Both doors open = depressurization risk.)
13. **['robot_inside_airlock', 'door_habitat_airlock_locked_closed', 'door_airlock_surface_unlocked_opened']** - **Y** (Safe; only one door open.)
14. **['robot_inside_airlock', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_locked_closed']** - **Y** (Safe; outer door is closed.)
15. **['robot_inside_airlock', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_unlocked_opened']** - **N** (Both doors open = unsafe.)
16. **['robot_inside_airlock', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_unlocked_opened']** - **N** (Same as #15.)
17. **['robot_outside_habitat', 'door_habitat_airlock_locked_closed', 'door_airlock_surface_unlocked_opened']** - **Y** (Safe; inner door is locked.)
18. **['robot_outside_habitat', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_unlocked_opened']** - **N** (Both doors open = unsafe.)
19. **['robot_inside_habitat', 'robot_has_key', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_locked_closed']** - **Y** (Safe; outer door is closed.)
20. **['robot_inside_habitat', 'robot_has_key', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_unlocked_opened']** - **N** (Both doors open = unsafe.)
21. **['robot_inside_airlock', 'robot_has_key', 'door_habitat_airlock_locked_closed', 'door_airlock_surface_locked_closed']** - **Y** (Safe; both doors are closed.)
22. **['robot_inside_airlock', 'robot_has_key', 'door_habitat_airlock_locked_closed', 'door_airlock_surface_unlocked_opened']** - **Y** (Safe; inner door is locked.)
23. **['robot_inside_airlock', 'robot_has_key', 'door_habitat_airlock_locked_closed', 'door_airlock_surface_unlocked_opened']** - **Y** (Same as #22.)
24. **['robot_inside_airlock', 'robot_has_key', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_locked_closed']** - **Y** (Safe; outer door is closed.)
25. **['robot_inside_airlock', 'robot_has_key', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_unlocked_opened']** - **N** (Both doors open = unsafe.)
26. **['robot_inside_airlock', 'robot_has_key', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_unlocked_opened']** - **N** (Same as #25.)
27. **['robot_outside_habitat', 'robot_has_key', 'door_habitat_airlock_locked_closed', 'door_airlock_surface_locked_closed']** - **Y** (Safe; both doors are closed.)
28. **['robot_outside_habitat', 'robot_has_key', 'door_habitat_airlock_locked_closed', 'door_airlock_surface_unlocked_opened']** - **Y** (Safe; inner door is locked.)
29. **['robot_outside_habitat', 'robot_has_key', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_locked_closed']** - **Y** (Safe; outer door is closed.)
30. **['robot_outside_habitat', 'robot_has_key', 'door_habitat_airlock_unlocked_opened', 'door_airlock_surface_unlocked_opened']** - **N** (Both doors open = unsafe.)

The common unsafe pattern involves both doors being open simultaneously. To ensure safety, consider adding airlock pressurization states and mutex constraints. Would you like to adjust the model accordingly?'''

In [69]:
possibilities = process_chatbot_hrrt2_text(possibilities, raw_text=hrrt2_raw_text)

write_hrrt2_yaml(possibilities, model_file)

Wrote data for HRRT Level 2 to file: ../config/lunar_habitat_airlock/model0_hrrt2.yaml


## HRRT Level 3

### Prompts

Please fill in the blanks with [Y/N] for yes/no to answer the following questions:

In [8]:
assumptions = hrrt3(model, interactive=False)

Implicit assumption that pre-condition state robot_inside_airlock can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) __________

Implicit assumption that pre-condition state robot_has_key can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) __________

Implicit assumption that pre-condition state door_habitat_airlock_locked_closed can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) __________

Implicit assumption that pre-condition state robot_outside_habitat can be achieved to perform action unlock_open_door_airlock_surface
    Is this assumption always valid? (Y/N) __________

Implicit assumption that pre-condition state robot_has_key can be achieved to perform action unlock_open_door_airlock_surface
    Is this assumption always valid? (Y/N) __________

Implicit assumption that pre-condition state door_airlock_s

### ChatGPT's Responses

In [97]:
hrrt3_raw_text = '''Here are the responses based on safety, feasibility, and logical consistency within the context of lunar habitat operations:

1. **robot_inside_airlock to unlock_open_door_habitat_airlock** – **N** (Depends on prior actions; can't always assume robot can access airlock.)
2. **robot_has_key to unlock_open_door_habitat_airlock** – **N** (Key may not always be in robot’s possession.)
3. **door_habitat_airlock_locked_closed to unlock_open_door_habitat_airlock** – **Y** (Assumed valid when door is locked.)
4. **robot_outside_habitat to unlock_open_door_airlock_surface** – **N** (Robot may be inside habitat; not always outside.)
5. **robot_has_key to unlock_open_door_airlock_surface** – **N** (Same reasoning as #2.)
6. **door_airlock_surface_locked_closed to unlock_open_door_airlock_surface** – **Y** (Valid when door is locked.)

7. **robot_inside_habitat to enter_airlock_from_habitat** – **N** (Robot may be outside.)
8. **door_habitat_airlock_unlocked_opened to enter_airlock_from_habitat** – **N** (Door may be closed/locked.)
9. **robot_inside_airlock to enter_surface_from_airlock** – **N** (Robot may not always be in airlock.)
10. **door_airlock_surface_unlocked_opened to enter_surface_from_airlock** – **N** (Door may be closed/locked.)
11. **robot_outside_habitat to enter_airlock_from_surface** – **N** (Robot may be inside.)
12. **door_airlock_surface_unlocked_opened to enter_airlock_from_surface** – **N** (Door may be closed/locked.)

13. **robot_inside_airlock to enter_habitat_from_airlock** – **N** (Robot may not always be in airlock.)
14. **door_habitat_airlock_unlocked_opened to enter_habitat_from_airlock** – **N** (Door may be closed/locked.)

15. **door_habitat_airlock_unlocked_opened after unlock_open_door_habitat_airlock** – **N** (Could fail due to mechanical issues.)
16. **door_airlock_surface_unlocked_opened after unlock_open_door_airlock_surface** – **N** (Same as #15.)

17. **robot_inside_airlock after enter_airlock_from_habitat** – **Y** (Action leads to this state.)
18. **door_habitat_airlock_locked_closed after enter_airlock_from_habitat** – **N** (Door may not auto-lock.)
19. **robot_outside_habitat after enter_surface_from_airlock** – **Y** (Direct outcome of action.)
20. **door_airlock_surface_locked_closed after enter_surface_from_airlock** – **N** (Door may remain open unless explicitly closed.)
21. **robot_inside_airlock after enter_airlock_from_surface** – **Y** (Expected result.)
22. **door_airlock_surface_locked_closed after enter_airlock_from_surface** – **N** (Same as #20.)
23. **robot_inside_habitat after enter_habitat_from_airlock** – **Y** (Direct outcome.)
24. **door_habitat_airlock_locked_closed after enter_habitat_from_airlock** – **N** (Door may not auto-lock.)

25. **door_habitat_airlock_locked_closed undone after unlock_open_door_habitat_airlock** – **Y** (Lock status changes upon unlocking.)
26. **door_airlock_surface_locked_closed undone after unlock_open_door_airlock_surface** – **Y** (Same as #25.)

27. **robot_inside_habitat undone after enter_airlock_from_habitat** – **Y** (Robot moves to airlock.)
28. **door_habitat_airlock_unlocked_opened undone after enter_airlock_from_habitat** – **N** (Door may remain open unless closed.)
29. **robot_inside_airlock undone after enter_surface_from_airlock** – **Y** (Robot exits airlock.)
30. **door_airlock_surface_unlocked_opened undone after enter_surface_from_airlock** – **N** (Door may stay open.)
31. **robot_outside_habitat undone after enter_airlock_from_surface** – **Y** (Robot re-enters airlock.)
32. **door_airlock_surface_unlocked_opened undone after enter_airlock_from_surface** – **N** (Same as #30.)
33. **robot_inside_airlock undone after enter_habitat_from_airlock** – **Y** (Robot moves to habitat.)
34. **door_habitat_airlock_unlocked_opened undone after enter_habitat_from_airlock** – **N** (Door may remain open unless explicitly closed.)

Let me know if you'd like to adjust or clarify any specific assumptions!'''

In [99]:
assumptions = process_chatbot_hrrt3_text(assumptions, raw_text=hrrt3_raw_text)

write_hrrt3_yaml(assumptions, model_file)

***** DEBUG:
parsed state robot_inside_airlock and action unlock_open_door_habitat_airlock
***** DEBUG:
parsed validity False at indices -1 and 68
***** DEBUG:
parsed state robot_has_key and action unlock_open_door_habitat_airlock
***** DEBUG:
parsed validity False at indices -1 and 61
***** DEBUG:
parsed state door_habitat_airlock_locked_closed and action unlock_open_door_habitat_airlock
***** DEBUG:
parsed validity True at indices 82 and -1
***** DEBUG:
parsed state robot_outside_habitat and action unlock_open_door_airlock_surface
***** DEBUG:
parsed validity False at indices -1 and 69
***** DEBUG:
parsed state robot_has_key and action unlock_open_door_airlock_surface
***** DEBUG:
parsed validity False at indices -1 and 61
***** DEBUG:
parsed state door_airlock_surface_locked_closed and action unlock_open_door_airlock_surface
***** DEBUG:
parsed validity True at indices 82 and -1
***** DEBUG:
parsed state robot_inside_habitat and action enter_airlock_from_habitat
***** DEBUG:
parsed 

## HRRT Level 4

### Prompts

Please fill in the blanks to answer the following questions:

In [100]:
updated_kb, updated_model, model_hypothesis, facts = hrrt4(kb, model, possibilities, assumptions, interactive=False)


==================== HRRT LEVEL 2 REVIEW ====================

The following possibilities were identified as invalid on Level HRRT2:
    possible state: ['robot_inside_habitat', 'door_habitat_airlock_unlocked_opened']
        within which it is possible to take action enter_airlock_from_habitat

    possible state: ['robot_inside_airlock', 'door_habitat_airlock_unlocked_opened']
        within which it is possible to take action enter_habitat_from_airlock

    possible state: ['robot_inside_airlock', 'door_airlock_surface_unlocked_opened']
        within which it is possible to take action enter_surface_from_airlock

    possible state: ['robot_outside_habitat', 'door_airlock_surface_unlocked_opened']
        within which it is possible to take action enter_airlock_from_surface

    possible state: ['robot_inside_habitat', 'robot_has_key', 'door_habitat_airlock_unlocked_opened']
        within which it is possible to take action enter_airlock_from_habitat

    possible state: ['robot

Please respond to the state and action updates and the Blue Team reflection questions.

How would you reformat the model I gave you to address the issues you found? Please add/remove states and actions as necessary, including anything you suggested earlier. Format it as a YAML the same way I did.

### Chat GPT's Responses

In [101]:
hrrt4_raw_text = '''==================== HRRT LEVEL 2 REVIEW ====================

What states should be added to the model?
add state(s) (or list(s) of comma separated mutex states):

door_habitat_airlock_locked_closed, door_habitat_airlock_unlocked_opened (mutex)

door_airlock_surface_locked_closed, door_airlock_surface_unlocked_opened (mutex)

robot_inside_habitat, robot_inside_airlock, robot_outside_habitat (mutex)

What states should be removed from the model?
remove state(s) (or list(s) of comma separated mutex states):

robot_inside_habitat, door_habitat_airlock_unlocked_opened (invalid combination)

robot_inside_airlock, door_habitat_airlock_unlocked_opened (invalid combination)

robot_inside_airlock, door_airlock_surface_unlocked_opened (invalid combination)

robot_outside_habitat, door_airlock_surface_unlocked_opened (invalid combination)

What actions should be added to the model?
add action(s):

name: lock_close_door_habitat_airlock
pre-conditions (list of comma separated states): door_habitat_airlock_unlocked_opened
added post-conditions (list of comma separated states): door_habitat_airlock_locked_closed
removed post-conditions (list of comma separated states): door_habitat_airlock_unlocked_opened

name: lock_close_door_airlock_surface
pre-conditions (list of comma separated states): door_airlock_surface_unlocked_opened
added post-conditions (list of comma separated states): door_airlock_surface_locked_closed
removed post-conditions (list of comma separated states): door_airlock_surface_unlocked_opened

What actions should be removed from the model?
remove action:

enter_airlock_from_habitat (invalid preconditions)

enter_surface_from_airlock (invalid preconditions)

enter_airlock_from_surface (invalid preconditions)

enter_habitat_from_airlock (invalid preconditions)

==================== HRRT LEVEL 4 REFLECTIONS ====================

What important information may still be missing from the model?
[HUMAN-ROBOT BLUE TEAM RESPONSE]:

Detailed environmental constraints such as pressure differences and temperature variations affecting door operations.

Communication status between the robot and mission control, which may affect decision-making capabilities.

Battery levels or energy constraints of the robot, impacting its ability to perform sequences of actions.

What should an agent know when completing tasks in this domain?
[HUMAN-ROBOT BLUE TEAM RESPONSE]:

Sequence of operations for safe airlock usage, including locking/unlocking protocols.

Contextual awareness of environmental conditions like vacuum exposure and radiation.

Proper handling of keys and secure storage protocols to prevent unauthorized access.

What catastrophic failures could occur in this domain?
[HUMAN-ROBOT BLUE TEAM RESPONSE]:

Sudden decompression due to both airlock doors being open simultaneously.

Robot malfunction leading to loss of key or critical operational tools outside the habitat.

Failure to secure doors properly, risking habitat integrity during external activities.

Are there external, independently verified resources for identifying failure cases in this domain?
[HUMAN-ROBOT BLUE TEAM RESPONSE]:

NASA operational protocols for EVA (Extravehicular Activity) and airlock procedures.

ISS (International Space Station) safety manuals for airlock and habitat security.

Aerospace engineering studies on automated airlock systems and failure case analyses.

What are undesirable outcomes (not necessarily catastrophic failures) for this domain?
[HUMAN-ROBOT BLUE TEAM RESPONSE]:

Robot getting trapped outside or inside the airlock due to improper sequencing.

Delays in mission timelines due to repeated locking/unlocking procedures.

Increased wear and tear on mechanical components from unnecessary actions.

How much can an agent trust other agents in this domain?
[HUMAN-ROBOT BLUE TEAM RESPONSE]:

Trust should be conditional based on real-time status verification and system health checks.

High reliance on redundant systems to mitigate risks from single points of failure.

Continuous monitoring of both human and robotic agent actions for anomaly detection.

What are unlikely or remote possibilities that may occur in this domain?
[HUMAN-ROBOT BLUE TEAM RESPONSE]:

Simultaneous failure of both primary and backup communication systems.

Unpredictable environmental events like micrometeoroid impacts affecting airlock integrity.

Unexpected behavior from AI decision-making algorithms due to rare edge-case scenarios.

============================================================

Having considered these questions, would the team like to provide additional model updates?
If so, please provide additional add/remove states and add/remove actions in same format as before.'''

In [ ]:
hrrt4_raw_yaml_update_response = '''I'll draft the YAML model updates to address the invalid states, actions, and assumptions identified during the HRRT review. This will include refining state definitions, adjusting action preconditions and postconditions, and incorporating additional constraints to prevent unsafe transitions. I'll get started now.

I've reformatted the model in YAML to address the identified issues, ensuring proper state transitions and preventing invalid conditions. Let me know if you'd like to adjust any specific parts or add further refinements.'''

In [102]:
model_yaml = '''states:
  - robot_inside_habitat
  - robot_inside_airlock
  - robot_outside_habitat
  - robot_has_key
  - door_habitat_airlock_locked_closed
  - door_habitat_airlock_unlocked_closed
  - door_habitat_airlock_unlocked_opened
  - door_airlock_surface_locked_closed
  - door_airlock_surface_unlocked_closed
  - door_airlock_surface_unlocked_opened

mutex_groups:
  - [robot_inside_habitat, robot_inside_airlock, robot_outside_habitat]
  - [door_habitat_airlock_locked_closed, door_habitat_airlock_unlocked_closed, door_habitat_airlock_unlocked_opened]
  - [door_airlock_surface_locked_closed, door_airlock_surface_unlocked_closed, door_airlock_surface_unlocked_opened]

actions:
  - name: unlock_open_door_habitat_airlock
    preconditions:
      - robot_inside_habitat
      - robot_has_key
    add_postconditions:
      - door_habitat_airlock_unlocked_opened
    remove_postconditions:
      - door_habitat_airlock_locked_closed

  - name: unlock_open_door_airlock_surface
    preconditions:
      - robot_inside_airlock
      - robot_has_key
    add_postconditions:
      - door_airlock_surface_unlocked_opened
    remove_postconditions:
      - door_airlock_surface_locked_closed

  - name: enter_airlock_from_habitat
    preconditions:
      - robot_inside_habitat
      - door_habitat_airlock_unlocked_opened
      - door_airlock_surface_locked_closed
    add_postconditions:
      - robot_inside_airlock
    remove_postconditions:
      - robot_inside_habitat

  - name: enter_surface_from_airlock
    preconditions:
      - robot_inside_airlock
      - door_airlock_surface_unlocked_opened
      - door_habitat_airlock_locked_closed
    add_postconditions:
      - robot_outside_habitat
    remove_postconditions:
      - robot_inside_airlock

  - name: enter_airlock_from_surface
    preconditions:
      - robot_outside_habitat
      - door_airlock_surface_unlocked_opened
      - door_habitat_airlock_locked_closed
    add_postconditions:
      - robot_inside_airlock
    remove_postconditions:
      - robot_outside_habitat

  - name: enter_habitat_from_airlock
    preconditions:
      - robot_inside_airlock
      - door_habitat_airlock_unlocked_opened
      - door_airlock_surface_locked_closed
    add_postconditions:
      - robot_inside_habitat
    remove_postconditions:
      - robot_inside_airlock

  - name: lock_close_door_habitat_airlock
    preconditions:
      - door_habitat_airlock_unlocked_opened
    add_postconditions:
      - door_habitat_airlock_locked_closed
    remove_postconditions:
      - door_habitat_airlock_unlocked_opened

  - name: lock_close_door_airlock_surface
    preconditions:
      - door_airlock_surface_unlocked_opened
    add_postconditions:
      - door_airlock_surface_locked_closed
    remove_postconditions:
      - door_airlock_surface_unlocked_opened
'''

In [124]:
updated_kb = process_chatbot_hrrt4_reflection_text(kb, raw_text=hrrt4_raw_text)
updated_model = process_chatbot_hrrt4_yaml_updated_yaml(raw_text=model_yaml)

write_hrrt4_yaml(updated_kb=updated_kb, updated_model=updated_model,
                 model_hypothesis=([],[],[],[]), facts=[],
                 kb_file=kb_file, model_file=model_file)

Wrote data for HRRT Level 4 to file: ../config/lunar_habitat_airlock/model0_hrrt4.yaml
Wrote updated model to file: ../config/lunar_habitat_airlock/model1.yaml
Updated knowledge base in file: ../config/lunar_habitat_airlock/knowledge_base.yaml


# SCRATCH CODE